## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [2]:
# Stopwords
sw = stopwords.words("english")

# functions for removing stopwords
#From previous
def remove_stop(text) :
    word_tokens = word_tokenize(text)
    tokens = [w for w in word_tokens if not w.lower() in sw]
    new_string = ' '.join(tokens)
    return(new_string)

In [3]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()
df = pd.read_sql("select * from conventions", convention_db)
df.head(3)

,party,night,speaker,speaker_count,time,text,text_len,file
0,Democratic,4,Unknown,1,00:00,Skip to content The Company Careers Press Free...,127,www_rev_com_blog_transcripts2020-democratic-na...
1,Democratic,4,Speaker 1,1,00:33,I’m here by calling the full session of the 48...,41,www_rev_com_blog_transcripts2020-democratic-na...
2,Democratic,4,Speaker 2,1,00:59,"Every four years, we come together to reaffirm...",17,www_rev_com_blog_transcripts2020-democratic-na...


### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [4]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute('''SELECT text, party FROM conventions''')

# store the results in convention_data
for row in query_results :
    row1 = remove_stop(row[0])
    row1 = row1.lower()
    row2 = row[1]
    new_list = [row1, row2]
    convention_data.append(new_list)    


Let's look at some random entries and see if they look right. 

In [5]:
random.choices(convention_data,k=10)

[['nevada .', 'Democratic'],
 ['enough enough . ’ time help small businesses . middle class folks manage way pandemic .',
  'Democratic'],
 ['teachers gwinnett county , georgia across country asked return classroom without plan keep safe parents exhausted juggling full-time work full-time childcare .',
  'Democratic'],
 ['faced president cowardice , joe biden man proven courage .', 'Democratic'],
 ['dreamer .', 'Democratic'],
 ['episcopal church , found ’ famous phrase kierkegaard , faith sees best dark . find one thing gives , ’ trying process . ’ trying convince share religious views . , ’ important gives reason hope purpose .',
  'Democratic'],
 ['important think faith way live life .', 'Democratic'],
 ['today , channel warrior roots battling human trafficking . ’ able lead largest trafficking prosecutions america , traveled foreign countries , working law enforcement ngos dismantle trafficking networks rescue people brutal conditions imaginable . young girls women sold sex slavery 

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [6]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2346 as features in the model.


In [7]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    
    ret_dict = dict()
    word_split = text.split()
    
    for word in word_split:
        if word in fw:
            ret_dict[word] = True

    return(ret_dict)

In [8]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [9]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [10]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [11]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [12]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     27.1 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     15.8 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                  defund = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

Many of the topic words are words you would expect to hear from the two political parties. Words such as religion, trade, drug and others are all topics heavily debated by the two parties. You would expect specific topics to be covered more or less by one party. For example, climate is usually a major talking point by the democratic party. 

After running the Naive Bayes model, the accuracy was only classifying tweets 50% of the time. If there are an even amount of democratic and republican tweets, then the model is equal to the baseline model and not offering significant improvement.

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [13]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [14]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [15]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.
for row in results:
    row3 = row[2].lower()
    row3 = str(row3)
    new_list = [row[1], row3]
    tweet_data.append(new_list)
    
tweet_data = [(party, conv_features(text,feature_words)) for (party, text) in tweet_data]

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [16]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [17]:
for party, tweet in tweet_data_sample :
    estimated_party = classifier.classify(tweet)
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: {'spoke': True, 'house': True, 'floor': True, 'protecting': True, 'health': True, 'care': True, 'women': True, 'work': True}
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: {}
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: {'trump': True, 'thinks': True, 'easy': True, 'students': True, 'burden': True, 'pay': True, 'student': True}
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: {'grateful': True, 'first': True, 'rescue': True, 'working': True, 'tirelessly': True, 'keep': True, 'people': True, 'provide': True, 'putting': True, 'lives': True}
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: {'make': True, 'even': True, 'greater': True}
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: {}
Actual party is Democratic and our classifer says Democratic.

H

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [18]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, pt in enumerate(tweet_data) :
    party, tweet = pt    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(tweet)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [19]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3424, 'Democratic': 854}),
             'Democratic': defaultdict(int,
                         {'Republican': 4498, 'Democratic': 1226})})

### Reflections

The model is classifiying results in favor of the republican party more so than the democratic. This may be partially because of the random seed used and those in the training set, but it is more likely that the words and topics covered in the most feautured list are more refelctive of the republican side. Topics such as "China" and "Freedom" may typically be covered more by the Republicans. 